In [1]:
import io
import os
import random
from pathlib import Path

In [2]:
%pwd

'c:\\Users\\Asus\\Machine_learning\\LLM\\Projects\\BIRD_benchmark_SQL\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Asus\\Machine_learning\\LLM\\Projects\\BIRD_benchmark_SQL'

In [5]:
import os
import json 
from Bird_bench_SQL import logger

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:
@dataclass(frozen=True)
class DataSplittingConfig:
    data_file_path  : Path
    train_file_path : Path
    test_file_path  : Path 
    db_id_name      : str 
    train_size      : int
    test_size       : int

In [8]:
from Bird_bench_SQL.constants import *
from Bird_bench_SQL.utils.common import read_yaml,create_directories
from Bird_bench_SQL.config.configuration import ConfigurationManager

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath    = CONFIG_FILE_PATH,
                 params_filepath    = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_splitting_config(self) -> DataSplittingConfig:
        config =  self.config.data_splitting
        params = self.params 
        logger.info('Data preprocessing initialized')
        data_procesing_config = DataSplittingConfig(data_file_path     = config.data_file_path,
                                                     train_file_path    = config.train_file_path,
                                                     test_file_path     = config.test_file_path,
                                                     db_id_name         = params.db_id,
                                                     train_size         = params.TRAIN_SIZE,
                                                     test_size          = params.TEST_SIZE) 
        return data_procesing_config        


In [10]:

manager = ConfigurationManager()
path = manager.get_data_splitting_config()
file_path = path.data_file_path

[2024-09-09 15:35:08,791: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-09 15:35:08,797: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-09 15:35:08,798: INFO: common: created directory at: artifacts]
[2024-09-09 15:35:08,798: INFO: 459710641: Data preprocessing initialized]


In [11]:
from Bird_bench_SQL.utils.common import load_json,save_json
import random

In [12]:
class DataSplitting:

    def __init__(self,config = DataSplittingConfig):
        self.config = config

    def data_splitting(self):

        if not os.path.exists(self.config.train_file_path):
            data_file       = self.config.data_file_path
            logger.info(f"Data splitting has started")
            data_file       = load_json(data_file)
            logger.info(f"{data_file} has loaded succesfully completed")
            specific_data   = []
            for data in data_file:
                if data['db_id'] == self.config.db_id_name:
                    specific_data.append(data)
            random.seed(42)
            random.shuffle(specific_data)
            total_size  = len(specific_data) 
            train_size  = int(0.7 * total_size)

            logger.info(f"{data_file} has loaded for data spiting")
            train_data  = specific_data[:train_size]
            test_data   = specific_data[train_size:]
            logger.info(f"{data_file} - data spiting completed")

            save_json(path  = self.config.train_file_path,
                      data  = train_data)
            save_json(path  = self.config.test_file_path,
                      data  = test_data)
            logger.info("data splitting completed")
        else:
            logger.info(f"{self.config.data_file_path} file is already present")


In [14]:
try:
    config                  = ConfigurationManager()
    data_splitting_config  = config.get_data_splitting_config()
    data_splitting          = DataSplitting(config=data_splitting_config)
    data_splitting.data_splitting()

except Exception as e:
    raise e

[2024-09-09 15:35:16,011: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-09 15:35:16,014: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-09 15:35:16,016: INFO: common: created directory at: artifacts]
[2024-09-09 15:35:16,016: INFO: 459710641: Data preprocessing initialized]
[2024-09-09 15:35:16,017: INFO: 3539417964: artifacts/data/train.json file is already present]
